In [1]:
!pip install --upgrade pip

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1


In [2]:
!pip install tensorflow pandas numpy

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [3]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16, MobileNetV3Small
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, F1Score
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.train import latest_checkpoint

2025-03-18 10:56:17.742963: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
model_name = "mobilenetv3small"
img_size = 256 # assume same for both width and height
batch_size = 8

data_dir = "data"
output_dir = "output"
# assume there is a json file of the same name inside these data subdirs.

checkpoint_dir = "checkpoints"
checkpoint_path = f"{checkpoint_dir}/{model_name}/" + "cp-{epoch:04d}.weights.h5"

In [67]:
df_train: pd.DataFrame = pd.read_csv(f"{data_dir}/uniface-ff-train.csv")
df_val: pd.DataFrame = pd.read_csv(f"{data_dir}/uniface-ff-test.csv")

In [68]:
df_train['label'] = df_train['label'].astype('str')
df_val['label'] = df_val['label'].astype('str')

In [69]:
train_datagen = ImageDataGenerator(
    rescale=(1./255),
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[-0.5, 0.5],
    rotation_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col = "filepath",
    y_col = "label",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    validate_filenames=True
)
val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col = "filepath",
    y_col = "label",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    validate_filenames=True
)

Found 22993 validated image filenames belonging to 1 classes.
Found 4479 validated image filenames belonging to 1 classes.


C:\Users\scout\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 11794 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(
C:\Users\scout\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 4465 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


In [70]:
pretrained_model = MobileNetV3Small(
    weights='imagenet',
    include_top=False,
    input_shape=(img_size, img_size, 3)
)

pretrained_model.trainable = False

# technique to "stack" layers, starting with pretrain model's layers
inputs = Input(shape=(img_size, img_size, 3))

cl = pretrained_model(inputs, training=False)

cl = GlobalAveragePooling2D()(cl)
cl = BatchNormalization()(cl)
cl = Dropout(0.2)(cl)

# this is the final layer; size must equal desired output size
outputs = Dense(1, activation='sigmoid')(cl)
model = Model(inputs, outputs)

model.summary(show_trainable=True)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                          ┃ Output Shape                  ┃        Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_7 (InputLayer)            │ (None, 256, 256, 3)           │              0 │    -    │
├───────────────────────────────────────┼───────────────────────────────┼────────────────┼─────────┤
│ vgg16 (Functional)                    │ (None, 8, 8, 512)             │     14,714,688 │    N    │
├───────────────────────────────────────┼───────────────────────────────┼────────────────┼─────────┤
│ global_average_pooling2d_3            │ (None, 512)                   │              0 │    -    │
│ (GlobalAveragePooling2D)              │                               │                │         │
├───────────────────────────────────────┼───────────────────────────────┼────────────────┼─────────┤
│ batch_normalization_2                 │ (None, 512)                   │          2,048 │    Y    │
│ (BatchNormalization)                  │                               │                │         │
├───────────────────────────────────────┼───────────────────────────────┼────────────────┼─────────┤
│ dropout_3 (Dropout)                   │ (None, 512)                   │              0 │    -    │
├───────────────────────────────────────┼───────────────────────────────┼────────────────┼─────────┤
│ dense_3 (Dense)                       │ (None, 1)                     │            513 │    Y    │
└───────────────────────────────────────┴───────────────────────────────┴────────────────┴─────────┘

 Total params: 14,717,249 (56.14 MB)

 Trainable params: 1,537 (6.00 KB)

 Non-trainable params: 14,715,712 (56.14 MB)

In [71]:
epochs = 10
learning_rate = 1e-6

model.compile(
    optimizer=Adam(learning_rate),
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy()]
)

In [72]:
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(os.getcwd(), checkpoint_path),
    save_weights_only=True,
    verbose=1
)

In [2]:
latest = latest_checkpoint(os.path.join(os.getcwd(), checkpoint_path))
print(latest)
#model.load_weights(latest)

NameError: name 'latest_checkpoint' is not defined

In [73]:
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=(train_generator.samples // train_generator.batch_size),
    validation_data=val_generator,
    validation_steps=(val_generator.samples // val_generator.batch_size),
    callbacks=[checkpoint_callback],
    verbose=1
)

C:\Users\scout\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2874/2874 ━━━━━━━━━━━━━━━━━━━━ 0s 664ms/step - binary_accuracy: 0.5362 - loss: 0.7131   
Epoch 1: saving model to D:\scout\Code\fakereal\models\checkpoints/vgg16/cp-0001.weights.h5
2874/2874 ━━━━━━━━━━━━━━━━━━━━ 2267s 789ms/step - binary_accuracy: 0.5362 - loss: 0.7131 - val_binary_accuracy: 0.5693 - val_loss: 0.8077
Epoch 2/10
   1/2874 ━━━━━━━━━━━━━━━━━━━━ 32:48 685ms/step - binary_accuracy: 0.5000 - loss: 0.7571

C:\Users\scout\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: saving model to D:\scout\Code\fakereal\models\checkpoints/vgg16/cp-0002.weights.h5
2874/2874 ━━━━━━━━━━━━━━━━━━━━ 348s 121ms/step - binary_accuracy: 0.5000 - loss: 0.7571 - val_binary_accuracy: 0.5686 - val_loss: 0.8086
Epoch 3/10
2874/2874 ━━━━━━━━━━━━━━━━━━━━ 0s 618ms/step - binary_accuracy: 0.6411 - loss: 0.6722   
Epoch 3: saving model to D:\scout\Code\fakereal\models\checkpoints/vgg16/cp-0003.weights.h5
2874/2874 ━━━━━━━━━━━━━━━━━━━━ 2104s 732ms/step - binary_accuracy: 0.6412 - loss: 0.6722 - val_binary_accuracy: 0.6713 - val_loss: 0.6240
Epoch 4/10
   1/2874 ━━━━━━━━━━━━━━━━━━━━ 28:15 590ms/step - binary_accuracy: 0.6250 - loss: 0.6251
Epoch 4: saving model to D:\scout\Code\fakereal\models\checkpoints/vgg16/cp-0004.weights.h5
2874/2874 ━━━━━━━━━━━━━━━━━━━━ 329s 114ms/step - binary_accuracy: 0.6250 - loss: 0.6251 - val_binary_accuracy: 0.6729 - val_loss: 0.6213
Epoch 5/10
2874/2874 ━━━━━━━━━━━━━━━━━━━━ 0s 607ms/step - binary_accuracy: 0.7286 - loss: 0.6318   
Epoch 5: sa

In [ ]:
model.evaluate(val_generator, verbose=1)

In [74]:
model.save(f"{output_dir}/deepfake-{model_name}.keras")